In [39]:
import sys
import os
import json
import pandas as pd

output_directory = "/dfs/scratch0/maxlam/mmtl_share/feb_16_output"

In [46]:
nan = float("nan")

def get_filepath_with_substring(filenames, substring_match):
    for filename in filenames:
        if substring_match in filename:
            return filename
    return None

def extract_text(filepath):
    with open(filepath, "r") as f:
        return f.read()
    
def extract_scores_from_stdout(stdout):
    # Assume second to last line contains json data of scores
    lines = stdout.splitlines()
    try:
        d = eval(lines[-2])
    except:
        d = None
    return d

def gather_data(working_dir):
    data = []
    for dirname, dirnames, filenames in os.walk(working_dir):
        filenames = [os.path.join(dirname, filename) for filename in filenames]
        stderr_filepath = get_filepath_with_substring(filenames, "stderr")
        stdout_filepath = get_filepath_with_substring(filenames, "stdout")
        config_filepath = get_filepath_with_substring(filenames, "config")
        if stderr_filepath is not None and stdout_filepath is not None and config_filepath is not None:
            print("Processing: %s" % dirname)
            stdout_text = extract_text(stdout_filepath)
            stderr_text = extract_text(stderr_filepath)
            data.append({
                "stderr" : stderr_text,
                "stdout" : stdout_text,
                "config" : json.loads(extract_text(config_filepath)),
                "config_path" : config_filepath,
                "scores" : extract_scores_from_stdout(stdout_text)
            })
    return data
  
data = gather_data(output_directory)

Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/1
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/6
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/2
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/8
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/5
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/7
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/0
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/4
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/9
Processing: /dfs/scratch0/maxlam/mmtl_share/feb_16_output/3


In [48]:
def extract_keys(score):
    d = []
    for k,v in score.items():
        for k2,v2 in v.items():
            d.append((k,k2))
    return d

def extract_score(key, data_element):
    if data_element is None:
        return None
    high_level_key, inner_key = key
    return data_element[high_level_key][inner_key]

def extract_extra_value(k, data_dict):
    if k == "config_path":
        return data_dict["config_path"]
    
def aggregate_data(data):
    keys = extract_keys(data[0]["scores"])
    extra_keys = ["config_path"]
    aggregated_data = {k:[] for k in keys+extra_keys}    
    for d in data:
        for k in keys:
            aggregated_data[k].append(extract_score(k, d["scores"]))
        for k in extra_keys:
            aggregated_data[k].append(extract_extra_value(k, d))
    return aggregated_data
        
pd.DataFrame.from_dict(aggregate_data(data))

,"(COLA, COLA/train/accuracy)","(COLA, COLA/train/matthews_corr)","(COLA, COLA/valid/accuracy)","(COLA, COLA/valid/matthews_corr)","(COLA, COLA/test/accuracy)","(COLA, COLA/test/matthews_corr)","(SST2, SST2/train/accuracy)","(SST2, SST2/valid/accuracy)","(SST2, SST2/test/accuracy)","(MNLI, MNLI/train/accuracy)","(MNLI, MNLI/valid/accuracy)","(MNLI, MNLI/test/accuracy)","(RTE, RTE/train/accuracy)","(RTE, RTE/valid/accuracy)","(RTE, RTE/test/accuracy)","(WNLI, WNLI/train/accuracy)","(WNLI, WNLI/valid/accuracy)","(WNLI, WNLI/test/accuracy)","(QQP, QQP/train/accuracy)","(QQP, QQP/train/f1)","(QQP, QQP/train/acc_f1)","(QQP, QQP/valid/accuracy)","(QQP, QQP/valid/f1)","(QQP, QQP/valid/acc_f1)","(QQP, QQP/test/accuracy)","(QQP, QQP/test/f1)","(QQP, QQP/test/acc_f1)","(MRPC, MRPC/train/accuracy)","(MRPC, MRPC/train/f1)","(MRPC, MRPC/train/acc_f1)","(MRPC, MRPC/valid/accuracy)","(MRPC, MRPC/valid/f1)","(MRPC, MRPC/valid/acc_f1)","(MRPC, MRPC/test/accuracy)","(MRPC, MRPC/test/f1)","(MRPC, MRPC/test/acc_f1)","(STSB, STSB/train/spearman_corr)","(STSB, STSB/train/pearson_corr)","(STSB, STSB/train/pearson_spearman)","(STSB, STSB/valid/spearman_corr)","(STSB, STSB/valid/pearson_corr)","(STSB, STSB/valid/pearson_spearman)","(STSB, STSB/test/spearman_corr)","(STSB, STSB/test/pearson_corr)","(STSB, STSB/test/pearson_spearman)","(QNLI, QNLI/train/accuracy)","(QNLI, QNLI/valid/accuracy)","(QNLI, QNLI/test/accuracy)",config_path
0,0.499854,0.002202,0.487434,-0.023580,0.503356,0.006073,0.498692,0.500074,0.501147,0.334001,0.331419,0.329638,0.502510,0.510040,0.490975,0.501969,0.417323,0.563380,0.498816,0.423497,0.461157,0.502556,0.425239,0.463898,0.500841,0.424502,0.462671,0.483129,0.552932,0.518030,0.509804,0.596774,0.553289,0.502609,0.570140,0.536374,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.497995,0.501122,0.505583,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/1/config
1,0.706725,0.000000,0.694915,0.000000,0.691275,0.000000,0.557230,0.560208,0.509174,0.333100,0.334246,0.355167,0.508534,0.473896,0.527076,0.494094,0.480315,0.436620,0.630238,0.000000,0.315119,0.632431,0.000000,0.316215,0.631833,0.000000,0.315916,0.668712,0.801471,0.735091,0.702206,0.825054,0.763630,0.664928,0.798747,0.731837,0.150954,0.161718,0.156336,0.177686,0.199868,0.188777,0.017749,-0.006247,0.005751,0.499248,0.502745,0.505400,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/6/config
2,0.706725,0.000000,0.694915,0.000000,0.691275,0.000000,0.779060,0.767409,0.736239,0.623276,0.625941,0.617880,0.534639,0.489960,0.552347,0.494094,0.480315,0.436620,0.768283,0.730886,0.749585,0.767638,0.728905,0.748271,0.771308,0.732512,0.751910,0.699693,0.780049,0.739871,0.714461,0.798268,0.756365,0.693913,0.771231,0.732572,-0.252922,-0.217812,-0.235367,-0.192050,-0.147731,-0.169890,0.006779,0.015158,0.010969,0.735244,0.731013,0.749771,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/2/config
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/8/config
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/5/config
5,0.708772,0.066222,0.696084,0.051630,0.694151,0.080368,0.917500,0.912324,0.899083,0.789474,0.779402,0.735354,0.637550,0.594378,0.606498,0.517717,0.440945,0.450704,0.868740,0.830066,0.849403,0.863103,0.821916,0.842510,0.861415,0.819799,0.840607,0.745092,0.825310,0.785201,0.792892,0.861815,0.827354,0.761739,0.835402,0.798571,0.113238,0.106531,0.109884,0.138316,0.122513,0.130415,0.119083,0.116645,0.117864,0.860742,0.851306,0.866191,/dfs/scratch0/maxlam/mmtl_share/feb_16_output/7/config
6,0.706725,0.000000,0.694915,0.000000,0.691275,0.000000,0.557230,0.560208,0.509174,0.333530,0.332540,0.312754,0.491466,0.526104,0.472924,0.494094,0.4803